In [1]:
from selenium.webdriver.common.by import By
import requests
import re
import pandas as pd
import numpy as np
import datetime
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import warnings
warnings.filterwarnings('ignore')
import sqlite3
from bs4 import BeautifulSoup, BeautifulStoneSoup
import json
user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('user-agent={0}'.format(user_agent))
# options =  webdriver.Chrome('/home/porter/NOTEBOOK/chromedriver',options=options)
conn = sqlite3.connect("/home/buzzportal/crawling/opinion_outside.db")
cur = conn.cursor()

client_id = "l5s8tg1g0w" # 개발자센터에서 발급받은 Client ID 값
client_secret = "RPNXQKCp8erMRW4KvKWsME5m3F9QOYVJTZ8Lvj56" # 개발자센터에서 발급받은 Client Secret 값

def get_sentiment(content):
    printout = True
    url="https://naveropenapi.apigw.ntruss.com/sentiment-analysis/v1/analyze"

    headers = {
        "X-NCP-APIGW-API-KEY-ID": client_id,
        "X-NCP-APIGW-API-KEY": client_secret,
        "Content-Type": "application/json"
    }

    data = {
      "content": content
    }

    response = requests.post(url, data=json.dumps(data), headers=headers)
    rescode = response.status_code

    if(rescode == 200):        
        sentiment = json.loads(response.text)['document']['sentiment'] 
        pos_score = json.loads(response.text)['document']['confidence']['positive']
        neg_score = json.loads(response.text)['document']['confidence']['negative']
        neu_score = json.loads(response.text)['document']['confidence']['neutral']
#         if printout:
#             print("\n=== 감성분석 ===")
#             print("판정: {}\n점수: 긍정 {}, 중립 {}, 부정 {}".format(sentiment, pos_score, neu_score, neg_score)) 
#         return sentiment, pos_score, neu_score, neg_score
        return pos_score

    else:
        print("Error : " + response.text)
        return np.nan     
    
    

In [2]:
   

CNT = 3
cnt = CNT

url = "https://www.teamblind.com/kr/topics/주식·투자"
driver = webdriver.Chrome('/home/buzzportal/crawling/chromedriver',options=options)
driver.get(url)

while cnt:
#         print(cnt)
    driver.find_element(By.TAG_NAME,'body').send_keys(Keys.PAGE_DOWN)

#     driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)
    cnt -= 1

bs_obj = BeautifulStoneSoup(driver.page_source)
div = bs_obj.find('div', {'class': 'article-list '})
posts = div.find_all('div', {'class' : 'tit'})
posts_title = []
for t in posts:
    try:
        posts_title.append(t.find('a').find('span').decompose().text.strip())
    except:
        posts_title.append(t.find('a').text.strip())
posts_href = []
for l in posts:
    tmp = l.find('a')['href']
    posts_href.append(tmp[tmp.rfind('-'):])
posts_title = [re.sub(r'[~()%":\'/,!?]', '', i).replace(' ', '-').replace('\n', '') for i in posts_title]

root_path = "https://www.teamblind.com/kr/post/"
list_url = list()
list_url2 = list()
cnt = 0
#     print('처리 들어간다')

## 크롤링
for title, link in zip(posts_title, posts_href):
    # cnt += 1
    title2 = re.sub('[.]', '', title)
    list_url.append(root_path + title + link)
    list_url2.append(root_path + title2 + link)
url_dirs = pd.DataFrame({'url_raw' : list_url, 'url' : list_url2})
#     url_dirs.to_csv('urls.csv')
data = []
for i in range(len(url_dirs)):
#     print(i ," // ", len(url_dirs), "// ", url_dirs.loc[i,'url'] )
    headers = {'User-Agent': 'Mozilla/5.0'} 
    response = requests.get(url_dirs.loc[i,'url'],  headers=headers)
    response

    bs_obj = BeautifulSoup(response.text)
    title = bs_obj.find('div', {'class' : 'article-view-head'})
    main = bs_obj.find('div', {'class' : 'article-view-contents'})
    like = bs_obj.find('div', {'class' : 'article_info'})
    comment = bs_obj.find('div', {'class' : 'article-comments'})
    try:
        ti = title.find('h2').text # 제목
        da = title.find('span', {'class' : 'date'}).text.replace('작성일', '') # 날짜
        if '일' in da:
            tmp = int(da.replace('일', ''))
            da = (datetime.datetime.today().date() - datetime.timedelta(days = tmp)).strftime('%Y%m%d')
        elif '어제' in da:
            da = (datetime.datetime.today().date() - datetime.timedelta(days = 1)).strftime('%Y%m%d')
        elif '분' in da:
            da = (datetime.datetime.today().date() - datetime.timedelta(days = 0)).strftime('%Y%m%d')
        elif '시간' in da:
            da = (datetime.datetime.today().date() - datetime.timedelta(days = 0)).strftime('%Y%m%d')
        elif '초' in da:
            da = (datetime.datetime.today().date() - datetime.timedelta(days = 0)).strftime('%Y%m%d')

        lo = title.find('span', {'class' : 'pv'}).text.replace('조회수', '') # 조회수

        tmp = main.find('p', {'class' : 'contents-txt'}).text # 본문
        try:
            tmp = tmp[:tmp.index('  ')]
        except:
            pass
        tmp

        try:
            li = like.find('a', {'class' : 'like'}).text.replace('좋아요', '') # 좋아요 수
            if not li:
                li = 0
        except:
            li = 0

        cc = comment.find('h3').text.replace('댓글 ', '') # 댓글 갯수

        if '0' == cc: # 댓글
            co = 0
        else:
            co = ''
            for c in comment.find_all('p', {'class' : 'cmt-txt'}):
                c = c.text
                try:
                    co += (c[:c.index('작성일')] + '\n')
                except:
                    co += (c + '\n')

        dict_data = {'date':da, 'title' : ti, 'main' : tmp, 'comments' : co, 'looks_count' : lo, 'likes_count' : li, 'comments_count' : cc}
    #     print(dict_data)
        data.append(dict_data)

    except:
        print("SPECIAL CHARACTER IN TITLE")    
    time.sleep(0.5)

    
df_ = pd.DataFrame(data).iloc[2:].reset_index(drop=True)

df_['DATETIME'] = (datetime.datetime.now() ).strftime('%Y%m%d-%H%M%S')
df_['VIEWS'] = df_['looks_count']
df_['LIKES'] = df_['likes_count']
df_['TITLE'] =  df_['title']
df_['CONTENT'] = df_['main']
df_['WRITER'] = np.nan

df_['SOURCE'] = "블라인드_주식투자"
df_['RELATED_ITEM'] = np.nan
df_['POSITIVITY'] = np.nan
for i in range(len(df_)):
    df_.loc[i,"POSITIVITY"] = get_sentiment(df_.loc[i,"TITLE"])
    
df_['OTHERS'] = df_[['comments_count']].to_dict('records')
df_['OTHERS'] = df_['OTHERS'].astype(str)
df_['NUM_REPLY'] = np.nan
df_['CRAWLING_DATETIME'] = (datetime.datetime.now() ).strftime('%Y%m%d-%H%M%S')
df_['URL'] =  np.nan
df_ = df_[['SOURCE','RELATED_ITEM',"DATETIME","TITLE","CONTENT","POSITIVITY","NUM_REPLY",'URL',"CRAWLING_DATETIME","OTHERS"]]
df_

df_new = df_[['SOURCE','RELATED_ITEM',"DATETIME","TITLE","CONTENT","POSITIVITY","NUM_REPLY",'URL',"CRAWLING_DATETIME","OTHERS"]]
df_new['YYYYMMDD'] = df_new['DATETIME'] .str[:8]
df_new

## 기존 DF에서 겹치는꺼 빼야하니 기존꺼ㅏ 불러오기

df_already = pd.read_sql("SELECT * FROM TABLE_BUZZ where SOURCE = '블라인드_주식투자'",conn)
df_already
df_already['YYYYMMDD'] = df_already['DATETIME'] .str[:8]

if len(df_new.merge(df_already, on=['YYYYMMDD','TITLE'], how='left')['CONTENT_y'].dropna()) ==0:
    df_ttl = df_new.copy()
else:
    df_ttl = df_new.iloc[:df_new.merge(df_already, on=['YYYYMMDD','TITLE'], how='left')['CONTENT_y'].dropna().index[0]]
    
df_ttl = df_ttl.drop(['YYYYMMDD'],axis=1)
df_ttl

# if len(df_ttl) >0:
#     df_ttl = df_ttl.reset_index(drop=True)

#     df_ttl.to_sql('TABLE_BUZZ',conn,if_exists = 'append',index = False)
#     print((datetime.datetime.now() ).strftime('%Y%m%d-%H%M%S') , " SUCCESS  : ",len(df_ttl))

# else:
#     print((datetime.datetime.now() ).strftime('%Y%m%d-%H%M%S') , " NO DATA TO UPDATE")


,SOURCE,RELATED_ITEM,DATETIME,TITLE,CONTENT,POSITIVITY,NUM_REPLY,URL,CRAWLING_DATETIME,OTHERS
0,블라인드_주식투자,NaN,20221018-160934,모든 투자자가 손실,.,0.012329,NaN,NaN,20221018-160937,{'comments_count': '4'}
1,블라인드_주식투자,NaN,20221018-160934,카뱅 직원분들,뭐 기사에 6억 뭐 풀로 땡겼느니직원들 곡소리 난다고 하는데가족중이 카뱅 직원있는데...,0.027198,NaN,NaN,20221018-160937,{'comments_count': '8'}
2,블라인드_주식투자,NaN,20221018-160934,빚 얼마나 있냐?,집 관련 대출은 빼고 해보자난 차할부 + 신용대출 해서4천만원,0.003470,NaN,NaN,20221018-160937,{'comments_count': '14'}
3,블라인드_주식투자,NaN,20221018-160934,저점은 이미 찍힌게 맞음 ㅇㅇ,연말 따뜻하게 보내자 형들…,0.077961,NaN,NaN,20221018-160937,{'comments_count': '1'}
4,블라인드_주식투자,NaN,20221018-160934,삼성스팩 & 산돌 따상 가자 🚀🚀🚀,삼성네임드 믿는다 !! ~ 균등4주 확정산돌 비례청약 풀매수 ~,0.017860,NaN,NaN,20221018-160937,{'comments_count': '0'}
5,블라인드_주식투자,NaN,20221018-160934,두슬람!,애두라 나 아무래도 무릎에서 산거 같다 ^^너거들 얼른 사라~ 지금 종아리다 ㅋ가보자고!,0.026144,NaN,NaN,20221018-160937,{'comments_count': '4'}
6,블라인드_주식투자,NaN,20221018-160934,"나스닥 14,000까지 서서히 갑니다.",내년 상반기까지요.그 이후 보합입니다.전고점은 2~3년 걸릴거에요,0.166642,NaN,NaN,20221018-160937,{'comments_count': '3'}
7,블라인드_주식투자,NaN,20221018-160934,데 호항이다~~,아들아 전학가자~,0.007073,NaN,NaN,20221018-160937,{'comments_count': '0'}
8,블라인드_주식투자,NaN,20221018-160934,주식 정리할까..,앱등이로서 내가 애플 주주라는거에자부심 가지고 살았는데 예전같지않네,0.019656,NaN,NaN,20221018-160937,{'comments_count': '2'}
9,블라인드_주식투자,NaN,20221018-160934,만약 상승장이 시작된다면,이전 처럼 3300 근방은 찍을 수 있을까? 난 뭔가 2700 박스권 나올거 같은데,0.066316,NaN,NaN,20221018-160937,{'comments_count': '4'}


In [3]:
driver.close()

In [ ]:
driver.quit()

In [5]:
from selenium.webdriver.common.by import By
import requests
import re
import pandas as pd
import numpy as np
import datetime
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import warnings
warnings.filterwarnings('ignore')
import sqlite3
from bs4 import BeautifulSoup, BeautifulStoneSoup
import json
user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('window-size=1920,1080')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('user-agent={0}'.format(user_agent))
# options =  webdriver.Chrome('/home/porter/NOTEBOOK/chromedriver',options=options)
conn = sqlite3.connect("/home/buzzportal/crawling/opinion_outside.db")
cur = conn.cursor()

client_id = "l5s8tg1g0w" # 개발자센터에서 발급받은 Client ID 값
client_secret = "RPNXQKCp8erMRW4KvKWsME5m3F9QOYVJTZ8Lvj56" # 개발자센터에서 발급받은 Client Secret 값

from datetime import datetime, timedelta

def get_sentiment(content):
    printout = True
    url="https://naveropenapi.apigw.ntruss.com/sentiment-analysis/v1/analyze"

    headers = {
        "X-NCP-APIGW-API-KEY-ID": client_id,
        "X-NCP-APIGW-API-KEY": client_secret,
        "Content-Type": "application/json"
    }

    data = {
      "content": content
    }

    response = requests.post(url, data=json.dumps(data), headers=headers)
    rescode = response.status_code

    if(rescode == 200):        
        sentiment = json.loads(response.text)['document']['sentiment'] 
        pos_score = json.loads(response.text)['document']['confidence']['positive']
        neg_score = json.loads(response.text)['document']['confidence']['negative']
        neu_score = json.loads(response.text)['document']['confidence']['neutral']
#         if printout:
#             print("\n=== 감성분석 ===")
#             print("판정: {}\n점수: 긍정 {}, 중립 {}, 부정 {}".format(sentiment, pos_score, neu_score, neg_score)) 
#         return sentiment, pos_score, neu_score, neg_score
        return pos_score

    else:
        print("Error : " + response.text)
        return np.nan     
    
    
    
driver = webdriver.Chrome('/home/buzzportal/crawling/chromedriver',options=options)    
# driver = webdriver.Chrome('/home/lsi8505/chromedriver',options=options)
driver.get("https://www.youtube.com/c/SmartMoney0/videos")
time.sleep(10)

html= driver.page_source
html_res = BeautifulSoup(html, "html.parser") 
res_data_table =  html_res.find_all("ytd-grid-video-renderer")[1]
videos_l = []
for vid in range(len(html_res.find_all("ytd-grid-video-renderer"))):
    if ("1 day ago" in html_res.find_all("ytd-grid-video-renderer")[vid].find_all("span", {"class": "style-scope ytd-grid-video-renderer"})[1].text) & ("11" not in html_res.find_all("ytd-grid-video-renderer")[vid].find_all("span", {"class": "style-scope ytd-grid-video-renderer"})[1].text):
        
        sub_url = html_res.find_all("ytd-grid-video-renderer")[vid].find_all("a", {"class": "yt-simple-endpoint style-scope ytd-grid-video-renderer"})[0]['href']
        videos_l.append("https://www.youtube.com{}".format( sub_url))
videos_l



df_ttl = pd.DataFrame()
for vide in range(len(videos_l)):
    driver.get(videos_l[i])
    time.sleep(10)
    driver.execute_script("window.scrollTo(0, 1080)") 
    time.sleep(5)
    driver.execute_script("window.scrollTo(0, 500)") 
    time.sleep(1)
    
    html_source = driver.page_source
    soup = BeautifulSoup(html_source, 'html.parser')

    id_list = soup.select("div#header-author > h3 > #author-text > span")
    comment_list = soup.select("yt-formatted-string#content-text")

    id_final = []
    comment_final = []
    comment_list

    for i in range(len(comment_list)):
        temp_id = id_list[i].text
        temp_id = temp_id.replace('\n', '')
        temp_id = temp_id.replace('\t', '')
        temp_id = temp_id.replace('    ', '')
        id_final.append(temp_id) # 댓글 작성자

        temp_comment = comment_list[i].text
        temp_comment = temp_comment.replace('\n', '')
        temp_comment = temp_comment.replace('\t', '')
        temp_comment = temp_comment.replace('    ', '')
        comment_final.append(temp_comment) # 댓글 내용

    df_ = pd.DataFrame()
    df_['WRITER'] = id_final
    df_['DATETIME'] =(datetime.now() - timedelta(hours=15)).strftime('%Y%m%d-%H%M%S')
    df_['VIEWS'] = np.nan
    df_['LIKES'] = np.nan
    df_['TITLE'] =  comment_final
    df_['CONTENT'] = np.nan

    df_['SOURCE'] = "Youtube_스마트머니"
    df_['RELATED_ITEM'] = np.nan
    df_['POSITIVITY'] = np.nan
#     for i in range(len(df_)):
#         df_.loc[i,"POSITIVITY"] = get_sentiment(df_.loc[i,"TITLE"])

    df_['OTHERS'] = df_[['WRITER']].to_dict('records')
    df_['OTHERS'] = df_['OTHERS'].astype(str)
    df_['NUM_REPLY'] =np.nan
    df_['CRAWLING_DATETIME'] = (datetime.now() ).strftime('%Y%m%d-%H%M%S')
    df_['URL'] =  videos_l[i]
    df_
    df_ttl = df_ttl.append(df_)
driver.close()
df_ttl




df_new = df_ttl[['SOURCE','RELATED_ITEM',"DATETIME","TITLE","CONTENT","POSITIVITY","NUM_REPLY",'URL',"CRAWLING_DATETIME","OTHERS"]]
# df_new['YYYYMMDD'] = df_new['DATETIME'] .str[:8]
df_new

df_already = pd.read_sql("SELECT * FROM TABLE_BUZZ where SOURCE = 'Youtube_스마트머니'",conn)
df_already

df_final = pd.DataFrame() 
df_new = df_new[~df_new['URL'].isin(df_already['URL'])].reset_index(drop=True)
df_final = df_final.append(df_new)
df_final


if len(df_final) >0:

    df_final = df_final.reset_index(drop=True)
#     for i in range(len(df_final)):
#         df_final.loc[i,"POSITIVITY"] = get_sentiment(df_final.loc[i,"TITLE"])
    df_final
    df_final.to_sql('TABLE_BUZZ',conn,if_exists = 'append',index = False)
    print((datetime.now() ).strftime('%Y%m%d-%H%M%S')  , " Youtube_스마트머니 - SUCCESS  : ",len(df_final))

else:
    print((datetime.now() ).strftime('%Y%m%d-%H%M%S') , " Youtube_스마트머니 - NO DATA TO UPDATE")

IndexError: list index out of range

In [13]:
i

4

In [14]:
 videos_l#[i]

['https://www.youtube.com/watch?v=NiDxzx6glXU',
 'https://www.youtube.com/watch?v=wV2jy8N_eI8',
 'https://www.youtube.com/watch?v=2DL41zkmeos']

In [11]:
df_

,WRITER,DATETIME,VIEWS,LIKES,TITLE,CONTENT,SOURCE,RELATED_ITEM,POSITIVITY,OTHERS,NUM_REPLY,CRAWLING_DATETIME
0,문준영,20220929-233834,NaN,NaN,주린이 모두 고난의 어려운 시기입니다. 얼마나 인내해야 하는지를 가늠하기 위하여 앞...,NaN,Youtube_스마트머니,NaN,NaN,{'WRITER': ' 문준영'},NaN,20220930-143834
1,길거리탐험가,20220929-233834,NaN,NaN,원래 엄청난 악재인데 완화라??? 그것도 달러는 고강도 긴축인데 준기축은 완화라 좋...,NaN,Youtube_스마트머니,NaN,NaN,{'WRITER': ' 길거리탐험가'},NaN,20220930-143834
2,문준영,20220929-233834,NaN,NaN,,NaN,Youtube_스마트머니,NaN,NaN,{'WRITER': ' 문준영'},NaN,20220930-143834
3,hi ho!,20220929-233834,NaN,NaN,,NaN,Youtube_스마트머니,NaN,NaN,{'WRITER': ' hi ho!'},NaN,20220930-143834
4,핏brad,20220929-233834,NaN,NaN,아니야 곧 한국 망한다.,NaN,Youtube_스마트머니,NaN,NaN,{'WRITER': ' 핏brad'},NaN,20220930-143834


In [9]:
id_final

['  문준영', '  길거리탐험가', '  문준영', '  hi ho!', '  핏brad']

In [8]:
len(videos_l)

3

In [10]:
videos_l

['https://www.youtube.com/watch?v=NiDxzx6glXU',
 'https://www.youtube.com/watch?v=wV2jy8N_eI8',
 'https://www.youtube.com/watch?v=2DL41zkmeos']

In [7]:
df_

,WRITER,DATETIME,VIEWS,LIKES,TITLE,CONTENT,SOURCE,RELATED_ITEM,POSITIVITY,OTHERS,NUM_REPLY,CRAWLING_DATETIME
0,문준영,20220929-233834,NaN,NaN,주린이 모두 고난의 어려운 시기입니다. 얼마나 인내해야 하는지를 가늠하기 위하여 앞...,NaN,Youtube_스마트머니,NaN,NaN,{'WRITER': ' 문준영'},NaN,20220930-143834
1,길거리탐험가,20220929-233834,NaN,NaN,원래 엄청난 악재인데 완화라??? 그것도 달러는 고강도 긴축인데 준기축은 완화라 좋...,NaN,Youtube_스마트머니,NaN,NaN,{'WRITER': ' 길거리탐험가'},NaN,20220930-143834
2,문준영,20220929-233834,NaN,NaN,,NaN,Youtube_스마트머니,NaN,NaN,{'WRITER': ' 문준영'},NaN,20220930-143834
3,hi ho!,20220929-233834,NaN,NaN,,NaN,Youtube_스마트머니,NaN,NaN,{'WRITER': ' hi ho!'},NaN,20220930-143834
4,핏brad,20220929-233834,NaN,NaN,아니야 곧 한국 망한다.,NaN,Youtube_스마트머니,NaN,NaN,{'WRITER': ' 핏brad'},NaN,20220930-143834


In [6]:
videos_l

['https://www.youtube.com/watch?v=NiDxzx6glXU',
 'https://www.youtube.com/watch?v=wV2jy8N_eI8',
 'https://www.youtube.com/watch?v=2DL41zkmeos']

In [4]:
datetime.datetime.now()

datetime.datetime(2022, 9, 29, 19, 3, 47, 834919)

In [6]:
import os
os.chmod('/path/to/chromedriver', 0755) 

SyntaxError: leading zeros in decimal integer literals are not permitted; use an 0o prefix for octal integers (1894326064.py, line 2)

In [5]:
driver = webdriver.Chrome('/home/buzzportal/crawling/chromedriver',options=options)
driver.get("https://www.youtube.com/c/SmartMoney0/videos")
time.sleep(10)

WebDriverException: Message: 'chromedriver' executable may have wrong permissions. Please see https://chromedriver.chromium.org/home


In [15]:
# driver.execute_script("window.scrollTo(0, 1080)") 
# time.sleep(5)

In [16]:
html= driver.page_source
html_res = BeautifulSoup(html, "html.parser") 
res_data_table =  html_res.find_all("ytd-grid-video-renderer")[1]
videos_l = []
for vid in range(len(html_res.find_all("ytd-grid-video-renderer"))):
    if "days ago" in html_res.find_all("ytd-grid-video-renderer")[vid].find_all("span", {"class": "style-scope ytd-grid-video-renderer"})[1].text :#("1 day ago" in html_res.find_all("ytd-grid-video-renderer")[vid].find_all("span", {"class": "style-scope ytd-grid-video-renderer"})[1].text) & ("11" not in html_res.find_all("ytd-grid-video-renderer")[vid].find_all("span", {"class": "style-scope ytd-grid-video-renderer"})[1].text):
        
        sub_url = html_res.find_all("ytd-grid-video-renderer")[vid].find_all("a", {"class": "yt-simple-endpoint style-scope ytd-grid-video-renderer"})[0]['href']
        videos_l.append("https://www.youtube.com{}".format( sub_url))
videos_l

['https://www.youtube.com/watch?v=0UP0t-PksQ8',
 'https://www.youtube.com/watch?v=RjHK7jlfPcA',
 'https://www.youtube.com/watch?v=b20B107s_hE',
 'https://www.youtube.com/watch?v=8WiXYMRfZis',
 'https://www.youtube.com/watch?v=_VMkyYumBL0',
 'https://www.youtube.com/shorts/3IzPqG8JXb4',
 'https://www.youtube.com/watch?v=4cZHA1Igx_s',
 'https://www.youtube.com/watch?v=kTXHSqW9JQE',
 'https://www.youtube.com/watch?v=ZfBSfscW9z0',
 'https://www.youtube.com/watch?v=3qG0MiCTrig',
 'https://www.youtube.com/watch?v=QpRdFsBexVA',
 'https://www.youtube.com/watch?v=K2UA9UOnZhE',
 'https://www.youtube.com/watch?v=mrcNbPIgl7k',
 'https://www.youtube.com/watch?v=MJ0LNuXAp6k',
 'https://www.youtube.com/watch?v=0r9VqsR6Nqw',
 'https://www.youtube.com/watch?v=-_IqXmMicNQ',
 'https://www.youtube.com/watch?v=r5jjJsSPa6A',
 'https://www.youtube.com/watch?v=UEFaEkU_JkA',
 'https://www.youtube.com/watch?v=P1lGLtgJ3DM',
 'https://www.youtube.com/watch?v=gfRdep9CrcU',
 'https://www.youtube.com/watch?v=yR3b8uk

In [17]:
df_ttl = pd.DataFrame()
for i in range(len(videos_l)):
    driver.get(videos_l[i])
    time.sleep(10)
    driver.execute_script("window.scrollTo(0, 1080)") 
    time.sleep(5)
    driver.execute_script("window.scrollTo(0, 500)") 
    time.sleep(1)
    
    html_source = driver.page_source
    soup = BeautifulSoup(html_source, 'html.parser')

    id_list = soup.select("div#header-author > h3 > #author-text > span")
    comment_list = soup.select("yt-formatted-string#content-text")

    id_final = []
    comment_final = []
    comment_list

    for i in range(len(comment_list)):
        temp_id = id_list[i].text
        temp_id = temp_id.replace('\n', '')
        temp_id = temp_id.replace('\t', '')
        temp_id = temp_id.replace('    ', '')
        id_final.append(temp_id) # 댓글 작성자

        temp_comment = comment_list[i].text
        temp_comment = temp_comment.replace('\n', '')
        temp_comment = temp_comment.replace('\t', '')
        temp_comment = temp_comment.replace('    ', '')
        comment_final.append(temp_comment) # 댓글 내용

    df_ = pd.DataFrame()
    df_['WRITER'] = id_final
    df_['DATETIME'] =(datetime.now() - timedelta(hours=15)).strftime('%Y%m%d-%H%M%S')
    df_['VIEWS'] = np.nan
    df_['LIKES'] = np.nan
    df_['TITLE'] =  comment_final
    df_['CONTENT'] = np.nan

    df_['SOURCE'] = "Youtube_스마트머니"
    df_['RELATED_ITEM'] = np.nan
    df_['POSITIVITY'] = np.nan
#     for i in range(len(df_)):
#         df_.loc[i,"POSITIVITY"] = get_sentiment(df_.loc[i,"TITLE"])

    df_['OTHERS'] = df_[['WRITER']].to_dict('records')
    df_['OTHERS'] = df_['OTHERS'].astype(str)
    df_['NUM_REPLY'] =np.nan
    df_['CRAWLING_DATETIME'] = (datetime.now() + timedelta(hours=9)).strftime('%Y%m%d-%H%M%S')
    df_['URL'] =  videos_l[i]
    df_
    df_ttl = df_ttl.append(df_)
driver.close()
df_ttl

Error : {"status":400,"error":{"errorCode":"E001","message":"Unsupported empty or blank text"}}
Error : {"status":400,"error":{"errorCode":"E001","message":"Unsupported empty or blank text"}}


,WRITER,DATETIME,VIEWS,LIKES,TITLE,CONTENT,SOURCE,RELATED_ITEM,POSITIVITY,OTHERS,NUM_REPLY,CRAWLING_DATETIME,URL
0,이재웅,20220921-134826,NaN,NaN,"서상영 본부장님, 김석환 선임 연구원님, 박희찬 이사님 항상 매크로 관련하여 많이 ...",NaN,Youtube_스마트머니,NaN,99.974570,{'WRITER': ' 이재웅'},NaN,20220922-134827,https://www.youtube.com/watch?v=b20B107s_hE
1,문준영,20220921-134826,NaN,NaN,아주 반가운 시장 뉴스와 분석입니다,NaN,Youtube_스마트머니,NaN,99.960030,{'WRITER': ' 문준영'},NaN,20220922-134827,https://www.youtube.com/watch?v=b20B107s_hE
2,문준영,20220921-134826,NaN,NaN,,NaN,Youtube_스마트머니,NaN,NaN,{'WRITER': ' 문준영'},NaN,20220922-134827,https://www.youtube.com/watch?v=b20B107s_hE
0,미래에셋 스마트머니,20220921-134846,NaN,NaN,오늘 방송내용00:00 시작\r01:26 오프닝\r02:00 배터리 업황 점검\r0...,NaN,Youtube_스마트머니,NaN,0.027706,{'WRITER': ' 미래에셋 스마트머니'},NaN,20220922-134846,https://www.youtube.com/watch?v=b20B107s_hE
1,둉닝,20220921-134846,NaN,NaN,유익한 정보 고맙읍니 다 ∙.,NaN,Youtube_스마트머니,NaN,99.951645,{'WRITER': ' 둉닝'},NaN,20220922-134846,https://www.youtube.com/watch?v=b20B107s_hE
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,뭉게구름,20220921-135600,NaN,NaN,홍콩h증시 전망에 대해서 말씀 좀 해 주세요. 홍콩h증시 는 els에 많은 영향을 ...,NaN,Youtube_스마트머니,NaN,0.141361,{'WRITER': ' 뭉게구름'},NaN,20220922-135601,https://www.youtube.com/watch?v=3qG0MiCTrig
6,포메타,20220921-135600,NaN,NaN,이차전지 소재업체들,NaN,Youtube_스마트머니,NaN,0.029945,{'WRITER': ' 포메타'},NaN,20220922-135601,https://www.youtube.com/watch?v=3qG0MiCTrig
7,근찡,20220921-135600,NaN,NaN,편집자님 썸네일 무엇?! ㅋㅋㅋ 신경많이 쓰셨네여,NaN,Youtube_스마트머니,NaN,0.012819,{'WRITER': ' 근찡'},NaN,20220922-135601,https://www.youtube.com/watch?v=3qG0MiCTrig
8,김정열,20220921-135600,NaN,NaN,미하방가방가,NaN,Youtube_스마트머니,NaN,0.020892,{'WRITER': ' 김정열'},NaN,20220922-135601,https://www.youtube.com/watch?v=3qG0MiCTrig


In [24]:
df_new = df_ttl[['SOURCE','RELATED_ITEM',"DATETIME","TITLE","CONTENT","POSITIVITY","NUM_REPLY",'URL',"CRAWLING_DATETIME","OTHERS"]]
# df_new['YYYYMMDD'] = df_new['DATETIME'] .str[:8]
df_new

,SOURCE,RELATED_ITEM,DATETIME,TITLE,CONTENT,POSITIVITY,NUM_REPLY,URL,CRAWLING_DATETIME,OTHERS
0,Youtube_스마트머니,NaN,20220921-134826,"서상영 본부장님, 김석환 선임 연구원님, 박희찬 이사님 항상 매크로 관련하여 많이 ...",NaN,99.974570,NaN,https://www.youtube.com/watch?v=b20B107s_hE,20220922-134827,{'WRITER': ' 이재웅'}
1,Youtube_스마트머니,NaN,20220921-134826,아주 반가운 시장 뉴스와 분석입니다,NaN,99.960030,NaN,https://www.youtube.com/watch?v=b20B107s_hE,20220922-134827,{'WRITER': ' 문준영'}
2,Youtube_스마트머니,NaN,20220921-134826,,NaN,NaN,NaN,https://www.youtube.com/watch?v=b20B107s_hE,20220922-134827,{'WRITER': ' 문준영'}
0,Youtube_스마트머니,NaN,20220921-134846,오늘 방송내용00:00 시작\r01:26 오프닝\r02:00 배터리 업황 점검\r0...,NaN,0.027706,NaN,https://www.youtube.com/watch?v=b20B107s_hE,20220922-134846,{'WRITER': ' 미래에셋 스마트머니'}
1,Youtube_스마트머니,NaN,20220921-134846,유익한 정보 고맙읍니 다 ∙.,NaN,99.951645,NaN,https://www.youtube.com/watch?v=b20B107s_hE,20220922-134846,{'WRITER': ' 둉닝'}
...,...,...,...,...,...,...,...,...,...,...
5,Youtube_스마트머니,NaN,20220921-135600,홍콩h증시 전망에 대해서 말씀 좀 해 주세요. 홍콩h증시 는 els에 많은 영향을 ...,NaN,0.141361,NaN,https://www.youtube.com/watch?v=3qG0MiCTrig,20220922-135601,{'WRITER': ' 뭉게구름'}
6,Youtube_스마트머니,NaN,20220921-135600,이차전지 소재업체들,NaN,0.029945,NaN,https://www.youtube.com/watch?v=3qG0MiCTrig,20220922-135601,{'WRITER': ' 포메타'}
7,Youtube_스마트머니,NaN,20220921-135600,편집자님 썸네일 무엇?! ㅋㅋㅋ 신경많이 쓰셨네여,NaN,0.012819,NaN,https://www.youtube.com/watch?v=3qG0MiCTrig,20220922-135601,{'WRITER': ' 근찡'}
8,Youtube_스마트머니,NaN,20220921-135600,미하방가방가,NaN,0.020892,NaN,https://www.youtube.com/watch?v=3qG0MiCTrig,20220922-135601,{'WRITER': ' 김정열'}


In [28]:
df_already = pd.read_sql("SELECT * FROM TABLE_BUZZ where SOURCE = 'Youtube_스마트머니'",conn)
df_already

,SOURCE,RELATED_ITEM,DATETIME,TITLE,CONTENT,POSITIVITY,NUM_REPLY,URL,CRAWLING_DATETIME,OTHERS
0,Youtube_스마트머니,None,20220921-134826,"서상영 본부장님, 김석환 선임 연구원님, 박희찬 이사님 항상 매크로 관련하여 많이 ...",None,99.974570,None,https://www.youtube.com/watch?v=b20B107s_hE,20220922-134827,{'WRITER': ' 이재웅'}
1,Youtube_스마트머니,None,20220921-134826,아주 반가운 시장 뉴스와 분석입니다,None,99.960030,None,https://www.youtube.com/watch?v=b20B107s_hE,20220922-134827,{'WRITER': ' 문준영'}
2,Youtube_스마트머니,None,20220921-134826,,None,NaN,None,https://www.youtube.com/watch?v=b20B107s_hE,20220922-134827,{'WRITER': ' 문준영'}
3,Youtube_스마트머니,None,20220921-134846,오늘 방송내용00:00 시작\r01:26 오프닝\r02:00 배터리 업황 점검\r0...,None,0.027706,None,https://www.youtube.com/watch?v=b20B107s_hE,20220922-134846,{'WRITER': ' 미래에셋 스마트머니'}
4,Youtube_스마트머니,None,20220921-134846,유익한 정보 고맙읍니 다 ∙.,None,99.951645,None,https://www.youtube.com/watch?v=b20B107s_hE,20220922-134846,{'WRITER': ' 둉닝'}
...,...,...,...,...,...,...,...,...,...,...
70,Youtube_스마트머니,None,20220921-135600,홍콩h증시 전망에 대해서 말씀 좀 해 주세요. 홍콩h증시 는 els에 많은 영향을 ...,None,0.141361,None,https://www.youtube.com/watch?v=3qG0MiCTrig,20220922-135601,{'WRITER': ' 뭉게구름'}
71,Youtube_스마트머니,None,20220921-135600,이차전지 소재업체들,None,0.029945,None,https://www.youtube.com/watch?v=3qG0MiCTrig,20220922-135601,{'WRITER': ' 포메타'}
72,Youtube_스마트머니,None,20220921-135600,편집자님 썸네일 무엇?! ㅋㅋㅋ 신경많이 쓰셨네여,None,0.012819,None,https://www.youtube.com/watch?v=3qG0MiCTrig,20220922-135601,{'WRITER': ' 근찡'}
73,Youtube_스마트머니,None,20220921-135600,미하방가방가,None,0.020892,None,https://www.youtube.com/watch?v=3qG0MiCTrig,20220922-135601,{'WRITER': ' 김정열'}


In [29]:
df_final = pd.DataFrame() 
df_new = df_new[~df_new['URL'].isin(df_already['URL'])].reset_index(drop=True)
df_final = df_final.append(df_new)
df_final

,SOURCE,RELATED_ITEM,DATETIME,TITLE,CONTENT,POSITIVITY,NUM_REPLY,URL,CRAWLING_DATETIME,OTHERS


In [27]:
if len(df_final) >0:

    df_final = df_final.reset_index(drop=True)
#     for i in range(len(df_final)):
#         df_final.loc[i,"POSITIVITY"] = get_sentiment(df_final.loc[i,"TITLE"])
    df_final
    df_final.to_sql('TABLE_BUZZ',conn,if_exists = 'append',index = False)
    print((datetime.now() + timedelta(hours=9)).strftime('%Y%m%d-%H%M%S')  , " Youtube_스마트머니 - SUCCESS  : ",len(df_final))

else:
    print((datetime.now() + timedelta(hours=9)).strftime('%Y%m%d-%H%M%S') , " Youtube_스마트머니 - NO DATA TO UPDATE")

20220922-141553  Youtube_스마트머니 - SUCCESS  :  75
